# Capturing and Transmitting Data Using Computer Vision


### Libraries Used for This Project
1. <u>**OpenCV**</u> - is a computer vision and machine learning library that provides a wide range of tools and functions for image and video processing.
2. <u>**Numpy**</u> - is a Python library that provides powerful tools for working with multi-dimensional arrays and matrices, enabling efficient data manipulation.
3. <u>**Time**</u> - is a library providing tools for time manipulation, such as time measurement, program delay, and more.
4. <u>**OS**</u> - is a library providing various methods for interacting with the operating system.
5. <u>**Requests**</u> - is a Python library for making HTTP requests, allowing easy communication with web servers and APIs.
6. <u>**JSON**</u> - is a built-in Python library for encoding and decoding JSON data, commonly used for exchanging data between a server and a web application.
7. <u>**Datetime**</u> - is a built-in Python library providing classes for manipulating dates and times, offering functionalities such as formatting dates, calculating time differences, and more.

In [1]:
import cv2 as cv
import numpy as np
import requests
import json
from datetime import datetime
from dotenv import load_dotenv
import time
import os

load_dotenv()

True

### Method for Sending Data
1. It retrieves the endpoint from environment variables and gets the current time in UTC format.
2. Then, it prepares a payload containing the temperature value and current time.
3. It sets the headers to indicate JSON content.
4. Finally, it sends a POST request to the endpoint with the payload as JSON data.

In [2]:
def send_value(value):
    endpoint = os.getenv("ENDPOINT")
    current_time = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
    payload = {
        "Temperature": value,
        "Date": current_time
        }

    headers = {'Content-Type': 'application/json'}

    response = requests.post(endpoint, headers=headers, data=json.dumps(payload))

### Helper method for obtaining the diameter and position of the circle

In [3]:
def avg_circles(circles, b):
    avg_x = 0
    avg_y = 0
    avg_r = 0
    for i in range(b):
        avg_x += circles[0][i][0]
        avg_y += circles[0][i][1]
        avg_r += circles[0][i][2]
    avg_x = int(avg_x / (b))
    avg_y = int(avg_y / (b))
    avg_r = int(avg_r / (b))

    return avg_x, avg_y, avg_r

### Method for Obtaining the Distance Between Two Points in Two-Dimensional Space
- Utilizing the Pythagorean theorem in a circle

In [4]:
def dist_2_pts(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2) 

### Detection and Calibration of Captured Circles
1. Loads the specified frame.
2. Converts the image to grayscale.
3. Detects circular objects in the image using the HoughCircles method from the OpenCV library.
4. Computes the average values for the detected circles.
5. Draws circles and calibration markers on the image.
6. Allows the user to input desired values through visual representation.
7. Returns the minimum and maximum angle values, minimum and maximum values, units, and coordinates of the center and radius of the detected circle for further use in the program.

In [5]:
def calibrate_gauge(folder_path, filename):
	img = cv.imread(f"{folder_path}/{filename}.jpg")
	height, width = img.shape[:2]
	gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
	circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 20, np.array([]), 100, 50, int(height * 0.35), int(height * 0.48))
	a, b, c = circles.shape
	x,y,r = avg_circles(circles, b)

	cv.circle(img, (x, y), r, (0, 0, 255), 3, cv.LINE_AA)  
	cv.circle(img, (x, y), 2, (0, 255, 0), 3, cv.LINE_AA) 

	separation = 10.0
	interval = int(360 / separation)
	p1 = np.zeros((interval,2))  
	p2 = np.zeros((interval,2))
	p_text = np.zeros((interval,2))

	for i in range(0,interval):
		for j in range(0,2):
			if (j % 2 == 0):
				p1[i][j] = x + 0.9 * r * np.cos(separation * i * 3.14 / 180)
			else:
				p1[i][j] = y + 0.9 * r * np.sin(separation * i * 3.14 / 180)

	text_offset_x = 10
	text_offset_y = 5

	for i in range(0, interval):
		for j in range(0, 2):
			if (j % 2 == 0):
				p2[i][j] = x + r * np.cos(separation * i * 3.14 / 180)
				p_text[i][j] = x - text_offset_x + 1.2 * r * np.cos((separation) * (i + 9) * 3.14 / 180) 
			else:
				p2[i][j] = y + r * np.sin(separation * i * 3.14 / 180)
				p_text[i][j] = y + text_offset_y + 1.2 * r * np.sin((separation) * (i + 9) * 3.14 / 180)

	for i in range(0,interval):
		cv.line(img, (int(p1[i][0]), int(p1[i][1])), (int(p2[i][0]), int(p2[i][1])),(0, 255, 0), 2)
		cv.putText(img, '%s' %(int(i * separation)), (int(p_text[i][0]), int(p_text[i][1])), cv.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1, cv.LINE_AA)

	cv.imwrite(f"{folder_path}/{filename}-calibration.jpg", img)

	print (f"Created: {filename}-calibration.jpg")
	min_angle = input('Min angle (lowest possible angle of dial) - in degrees: ')
	max_angle = input('Max angle (highest possible angle) - in degrees: ') 
	min_value = input('Min value: ') 
	max_value = input('Max value: ') 
	units = input('Enter units: ')

	return min_angle, max_angle, min_value, max_value, units, x, y, r

### Obtaining and Printing Values from a Calibrated Image
1. Converts the image to grayscale using the cv.cvtColor() method.
2. Applies thresholding to the grayscale image using the cv.threshold() method.
3. Detects lines in the image using the HoughLinesP method from the OpenCV library.
4. Selects only those lines that are close to the center of the detected circle and have the correct orientation.
5. Determines the angle at which the pointer is displayed and converts it to a value within the defined range.
6. Returns the current value of the device based on the position of the pointer.

In [6]:
def get_current_value(img, min_angle, max_angle, min_value, max_value, x, y, r, folder_path,filename):
	gray2 = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

	thresh = 175
	maxValue = 255

	th, dst2 = cv.threshold(gray2, thresh, maxValue, cv.THRESH_BINARY_INV);

	minLineLength = 10
	maxLineGap = 0
	lines = cv.HoughLinesP(image=dst2, rho=3, theta=np.pi / 180, threshold=100,minLineLength=minLineLength, maxLineGap=0)

	final_line_list = []

	diff1LowerBound = 0.15 
	diff1UpperBound = 0.25
	diff2LowerBound = 0.5 
	diff2UpperBound = 1.0
	for i in range(0, len(lines)):
		for x1, y1, x2, y2 in lines[i]:
			diff1 = dist_2_pts(x, y, x1, y1) 
			diff2 = dist_2_pts(x, y, x2, y2) 
			if (diff1 > diff2):
				temp = diff1
				diff1 = diff2
				diff2 = temp
			if (((diff1<diff1UpperBound * r) and (diff1 > diff1LowerBound * r) and (diff2 < diff2UpperBound * r)) and (diff2 > diff2LowerBound * r)):
				line_length = dist_2_pts(x1, y1, x2, y2)
				final_line_list.append([x1, y1, x2, y2])

	if final_line_list:
		x1 = final_line_list[0][0]
		y1 = final_line_list[0][1]
		x2 = final_line_list[0][2]
		y2 = final_line_list[0][3]
		cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
	else:
		print("None")

	cv.imwrite(f"{folder_path}/{filename}-needle.jpg", img)

	dist_pt_0 = dist_2_pts(x, y, x1, y1)
	dist_pt_1 = dist_2_pts(x, y, x2, y2)
	if (dist_pt_0 > dist_pt_1):
		x_angle = x1 - x
		y_angle = y - y1
	else:
		x_angle = x2 - x
		y_angle = y - y2

	res = np.arctan(np.divide(float(y_angle), float(x_angle)))

	res = np.rad2deg(res)
	if x_angle > 0 and y_angle > 0:  # quadrant I
		final_angle = 270 - res
	if x_angle < 0 and y_angle > 0:  # quadrant II
		final_angle = 90 - res
	if x_angle < 0 and y_angle < 0:  # quadrant III
		final_angle = 90 - res
	if x_angle > 0 and y_angle < 0:  # quadrant IV
		final_angle = 270 - res

	old_min = float(min_angle)
	old_max = float(max_angle)

	new_min = float(min_value)
	new_max = float(max_value)

	old_value = final_angle

	old_range = (old_max - old_min)
	new_range = (new_max - new_min)
	new_value = (((old_value - old_min) * new_range) / old_range) + new_min

	return new_value

### Main method 
1. At the beginning, it opens a video file using cv.VideoCapture, which allows access to individual frames in the video.
2. It tries to create a directory named "Video" if it does not exist, so that it can save output frames.
3. Then it calibrates the gauge using the calibrate_gauge function, which obtains important parameters for measurement.
4. In an infinite loop, it reads individual frames from the video.
5. For each frame, it calls the get_current_value function, which determines the current value on the gauge based on calibration and the position of the pointer in the frame.
6. It prints the current value and sends it to desired location(I used PowerBI api).
7. Finally, it removes temporary files.

In [ ]:
def main():
    capture = cv.VideoCapture("gauge_sim.mp4")

    try:
        if not os.path.exists("Video"):
            os.makedirs("Video")
    except:
        print("ERROR creating directory")

    need_config = True
    while True:
        isTrue, frame = capture.read()

        name = "Video/frame.jpg"

        time.sleep(3)

        cv.imwrite(name, frame)

        if need_config:
            min_angle, max_angle, min_value, max_value, units, x, y, r = calibrate_gauge("Video", "frame")
            need_config = False

        time.sleep(3)

        img = cv.imread(name)
        value = get_current_value(img, min_angle, max_angle, min_value, max_value, x, y, r, "Video", "frame")
        print ("Current reading: %s %s" %(("%.2f" % value), units))

        time.sleep(3)

        send_value(value)

        os.remove(name)
        os.remove("Video/frame-needle.jpg")

        if cv.waitKey(20) == ord('d'):
            break

if __name__ == "__main__":
    main()